In [1]:
import requests
import config
from sqlalchemy import create_engine
import requests
from starwardob import starwardob # pip install starwar-dob

1074
948


In [2]:
# import postgres db credentials fron config file
password = config.password
user = config.user
db = config.db
port = config.port
host = config.host

In [3]:
# # create a connection to postgres database
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')
conn   = engine.raw_connection()
cursor = conn.cursor()

In [4]:
schema  = 'api_schema'
table = 'swapi_data'

In [5]:
engine.connect()

In [6]:
def process_api_data(schema, table):
    create_schema(schema)
    create_table(schema, table)
    truncate_table(schema, table)
    insert_data = extract_insert(schema, table)
    query_data = query_db(schema, table)
    if len(insert_data) == len(query_data):
        print(f"""Data count from api and database are {len(insert_data)} and {len(query_data)} respectively. Operation successfull!""")
    else:
        print("""Incomplete data transfer!""")

In [7]:
def create_schema(schema):
    cursor.execute(f""" CREATE SCHEMA IF NOT EXISTS {schema}; """)
    conn.commit()

In [8]:
def drop_table(schema, table):
    cursor.execute(f""" DROP TABLE IF EXISTS {schema}.{table} CASCADE; """)
    conn.commit()

In [9]:
def create_table(schema, table):
    cursor.execute(f"""   
    CREATE TABLE IF NOT EXISTS {schema}.{table}(
                    id SERIAL primary key not null
                    ,name        varchar(200)
                    ,height      varchar(50)
                    ,mass        varchar(50)
                    ,hair_color   varchar(200)
                    ,skin_color   varchar(200)
                    ,eye_color    varchar(200)
                    ,birth_year   varchar(200)
                    ,age          varchar(200)
                    ,gender       varchar(200)
                    ,homeworld    varchar(200)
                    ,films        int
                    ,species      varchar(200)
                    ,vehicles     int
                    ,starships    int
                    ,created      Timestamp
                    ,edited       Timestamp
                    ,url          varchar(200)
                    ,insert_date date DEFAULT now()
                        )
    """)
    conn.commit()

In [10]:
def truncate_table(schema, table):
    cursor.execute(f""" TRUNCATE TABLE {schema}.{table}; """)
    conn.commit()

In [11]:
def query_db(schema, table):
    cursor.execute(f""" select * from {schema}.{table} """)
    db_api_data = []
    for rec in cursor.fetchall():
        db_api_data.append(rec)
    return db_api_data

In [12]:
cursor.execute(f"""
                CREATE TABLE IF NOT EXISTS {schema}.swapi_data_raw(
                id SERIAL primary key,
                insert_date date DEFAULT now(),
                json_data jsonb
                )
                """)
conn.commit()

In [25]:
# len(api_data_list)
# new = {}
# for rec in api_data_list:
#     print(rec['mass'])
#     rec['mass'] = clean_str(rec['mass'])
#     print(rec['mass'])

len(api_data_list)
new = {}
for rec in api_data_list:
#     print(list(rec.items()))
    for key, value in list(rec.items()):
#         print(key, value)
        if key == 'mass':
            value = clean_str(value)
#         if key == 'birth_year':
#             age = star_war_age(value)   
#             value = age        

In [27]:
api_data_list

[{'name': 'Luke Skywalker',
  'height': '172',
  'mass': '77',
  'hair_color': 'blond',
  'skin_color': 'fair',
  'eye_color': 'blue',
  'birth_year': '19BBY',
  'gender': 'male',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/2/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['https://swapi.dev/api/vehicles/14/',
   'https://swapi.dev/api/vehicles/30/'],
  'starships': ['https://swapi.dev/api/starships/12/',
   'https://swapi.dev/api/starships/22/'],
  'created': '2014-12-09T13:50:51.644000Z',
  'edited': '2014-12-20T21:17:56.891000Z',
  'url': 'https://swapi.dev/api/people/1/'},
 {'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
 

In [14]:
url = "https://swapi.dev/api/people/"
api_data_list  = []
while url:
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        api_data_list.extend(data['results'])
    url  = response.json()['next']

    
    
#     for rec in data['results']:
#         rec_list = [
#             rec['name'],
#             rec['height'],
#   clean_str(rec['mass']),      
#             rec['hair_color'],
#             rec['skin_color'],
#             rec['eye_color'],
#             rec['birth_year'],
# star_war_age(rec['birth_year']),                    
#             rec['gender'],
#             rec['homeworld'],
#         len(rec['films']),
#         len(rec['species']),
#         len(rec['vehicles']),
#         len(rec['starships']),
#             rec['created'],
#             rec['edited'],
#             rec['url'],
#         ]
#         api_data_list.append(rec_list)
# url  = response.json()['next']


In [ ]:
for rec in data['results']:
    print(rec['mass'])

In [16]:
# function that converts to logical year
def star_war_age(year):
    if (year != 'unknown' and year != "n/a"):
        age = starwardob.StarwarDOB(year).age
    else:
        age = year
    return age

In [17]:
    test  = '1,324'
    

In [18]:
# function to remove unwanted character in a record
def clean_str(data):   
    data = str(data)
    clean_data = data.replace(",",'')
    return clean_data

In [19]:
clean_str(test)

'1324'

In [20]:
age = star_war_age('19BBY') 
age

71

In [21]:
# api_data_list

In [22]:
def extract_insert(schema, table):
    url = "https://swapi.dev/api/people/"
    api_data_list  = []
    while url:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            for rec in data['results']:
                rec_list = [
                    rec['name'],
                    rec['height'],
          clean_str(rec['mass']),      
                    rec['hair_color'],
                    rec['skin_color'],
                    rec['eye_color'],
                    rec['birth_year'],
       star_war_age(rec['birth_year']),                    
                    rec['gender'],
                    rec['homeworld'],
                len(rec['films']),
                    rec['species'],
                len(rec['vehicles']),
                len(rec['starships']),
                    rec['created'],
                    rec['edited'],
                    rec['url'],
                ]
                api_data_list.append(rec_list)
        url  = response.json()['next']
    #insert data into the database    
    for rec in api_data_list:
        cursor.execute(f""" 
              insert into {schema}.{table}(name, height, mass, hair_color, skin_color, eye_color, birth_year, age, gender, homeworld,
              films, species, vehicles, starships, created, edited, url) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", rec)
        conn.commit()
    return api_data_list

In [ ]:
if __name__ == '__main__':
    process_api_data(schema, table)

In [ ]:
# close cursor and connection
cursor.close()
conn.close()